In [29]:
import os
import subprocess
from datetime import datetime
from google.colab import drive

# === CẤU HÌNH RIÊNG (CHỈ THAY 1 LẦN) ===
USERNAME   = "FongNgoo"                                   # ← GitHub username
REPO_NAME  = "Basic_Dynamic_Prices_base_on_Demand_Model" # ← Tên repo
BRANCH     = "main"                                       # hoặc "master"
REPO_PATH  = f"/content/drive/MyDrive/Colab_Notebooks/{REPO_NAME}"
SSH_DIR    = "/content/drive/MyDrive/.colab_ssh"          # Nơi lưu key vĩnh viễn
# ======================================

REPO_DIR = f"/content/{REPO_NAME}"
REPO_URL = f"git@github.com:{USERNAME}/{REPO_NAME}.git"


In [28]:
def setup_permanent_ssh():
    """Chạy 1 lần duy nhất → tạo và lưu SSH key vĩnh viễn trên Drive"""
    drive.mount('/content/drive', force_remount=True)
    os.makedirs(SSH_DIR, exist_ok=True)

    key_path = f"{SSH_DIR}/id_colab"

    if os.path.exists(key_path):
        print("SSH key đã tồn tại trên Drive → sử dụng lại.")
    else:
        print("Đang tạo SSH key vĩnh viễn (lưu trên Drive)...")
        !ssh-keygen -t rsa -b 4096 -C "colab-permanent@$(date +%s)" -f "{key_path}" -N "" -q
        print("ĐÃ TẠO SSH KEY MỚI!")

    print("\n" + "="*70)
    print("COPY PUBLIC KEY DƯỚI ĐÂY VÀ DÁN VÀO: https://github.com/settings/ssh/new")
    print("="*70)
    !cat "{key_path}.pub"
    print("="*70)
    print("Sau khi dán xong → chạy load_ssh() để kích hoạt trong runtime này")
    print("Chỉ cần làm 1 lần duy nhất là xong mãi mãi!")

In [30]:
def load_ssh():
    """Tải SSH key từ Drive vào runtime (chạy mỗi khi mở notebook mới)"""
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive', force_remount=True)

    key_path = f"{SSH_DIR}/id_colab"
    if not os.path.exists(key_path):
        raise FileNotFoundError("Chưa có SSH key! Chạy setup_permanent_ssh() trước.")

    !mkdir -p /root/.ssh
    !cp "{key_path}" /root/.ssh/id_colab
    !cp "{key_path}.pub" /root/.ssh/id_colab.pub
    !chmod 600 /root/.ssh/id_colab
    !chmod 644 /root/.ssh/id_colab.pub

    # Cấu hình SSH dùng key này
    config = f"""
Host github.com
    HostName github.com
    User git
    IdentityFile /root/.ssh/id_colab
    StrictHostKeyChecking no
    """
    with open('/root/.ssh/config', 'w') as f:
        f.write(config.strip())

    # Fix host key
    !ssh-keyscan github.com >> /root/.ssh/known_hosts 2>/dev/null

    # Test kết nối
    print("Đang test SSH với GitHub...")
    result = subprocess.run(['ssh', '-T', 'git@github.com'], capture_output=True, text=True)
    if "successfully authenticated" in result.stderr:
        print("SSH KẾT NỐI THÀNH CÔNG! Hi {USERNAME}!")
    else:
        print("SSH chưa được add vào GitHub → chạy setup_permanent_ssh() để lấy public key")

In [31]:
def ensure_repo():
    """Clone repo nếu chưa có"""
    if not os.path.exists(REPO_PATH):
        print(f"Đang clone repo {USERNAME}/{REPO_NAME}...")
        !git clone "git@github.com:{USERNAME}/{REPO_NAME}.git" "{REPO_PATH}"
    os.chdir(REPO_PATH)
    print(f"Đã vào: {REPO_PATH}")

In [16]:
def push_only(message=None, force=False):
    """
    CHỈ PUSH – KHÔNG PULL, KHÔNG REBASE, KHÔNG HỎI
    Dùng khi bạn chắc chắn muốn đẩy hết mọi thứ trong Colab lên GitHub ngay lập tức
    """
    repo_path = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model"

    # 1. Fix Drive treo (bắt buộc)
    if not os.path.exists('/content/drive/MyDrive'):
        print("Drive chưa mount → mount lại...")
        drive.mount('/content/drive', force_remount=True)
    else:
        try:
            os.listdir('/content/drive/MyDrive')
        except:
            print("Drive bị treo → kill và remount...")
            !pkill -9 -f "drive" 2>/dev/null
            drive.mount('/content/drive', force_remount=True)

    # 2. Vào thư mục repo
    os.chdir(repo_path)
    print(f"Đã vào: {os.getcwd()}")

    # 3. Kiểm tra nhanh có gì thay đổi không
    status = subprocess.check_output(['git', 'status', '--porcelain']).decode().strip()
    if not status:
        print("Không có thay đổi nào → Không cần push.")
        return

    print(f"Phát hiện thay đổi:\n{status}")

    # 4. Commit
    msg = message or f"Quick push từ Colab - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    print(f"Đang commit: {msg}")
    !git add . --all
    !git commit -m "{msg}"

    # 5. Push (có lựa chọn force)
    if force:
        print("Đang FORCE PUSH (--force-with-lease)...")
        !git push origin {BRANCH} --force-with-lease
    else:
        print("Đang push bình thường...")
        result = subprocess.run(['git', 'push', 'origin', BRANCH])
        if result.returncode != 0:
            print("Push thường bị từ chối (có xung đột) → tự động chuyển sang force-with-lease")
            !git push origin {BRANCH} --force-with-lease

    print("PUSH XONG 100% LÊN GITHUB!")

In [24]:
fix_ssh_and_push()

^C
Mounted at /content/drive
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Đang test SSH...
Commit: Fix SSH & push - 2025-11-17 18:33:04
[main d782aa0] Fix SSH & push - 2025-11-17 18:33:04
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite AUTO_SYNC_COLAB_GITHUB_SSH.ipynb (100%)
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.

SSH HOẠT ĐỘNG ỔN ĐỊNH + ĐÃ PUSH XONG 100%!
Từ giờ chỉ cần gọi sync() hoặc push_only() bình thường là auto dùng SSH!


In [23]:
push_only()

Đã vào: /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
Phát hiện thay đổi:
M AUTO_SYNC_COLAB_GITHUB_SSH.ipynb
 M Basic_Dynamic_Prices_base_on_Demand_Model
Đang commit: Quick push từ Colab - 2025-11-17 18:32:49
[main e34259d] Quick push từ Colab - 2025-11-17 18:32:49
 1 file changed, 1 insertion(+), 1 deletion(-)
Đang push bình thường...
Push thường bị từ chối (có xung đột) → tự động chuyển sang force-with-lease
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
PUSH XONG 100% LÊN GITHUB!
